In [8]:
import pandas as pd

# Read dataframe
wide_df = pd.read_csv("../../data/processed/preprocessed_df.csv")

# Add binary outcome variable
wide_df["surg_engel_binary"] = wide_df["surg_engel"]==1

C:\Users\amaur\AppData\Local\Temp\ipykernel_6384\2834222239.py:4: DtypeWarning: Columns (80,122,145,312,316,320,324,380,458,507,518,519,520,521,522,523,524,561,567,583,584,585,586,587,596,597,599,600,603,615,616,617,622,628,631,632,634,641,677,690,693,707,709,713,715,716,718,720,721,723,727,728,730,743,745,752,754,756,757,759,763,765,767,769,771,786,805,807,808,809,810,812,813,882,883,891,900,963,970,971,979,988,1051,1058,1059,1067,1075,1076,1139,1146,1164,1227,1235,1252,1322,1323,1340,1411,1428,1518,1535,1553,1673,1725,1733,1785,1796,1814,1825,1877,1885,1948,1949,1958,1977,2029,2037,2089,2129,2181,2189,2241,2252,2253,2262,2281,2333,2341,2393,2405,2414,2422,2433,2485,2493,2545,2557,2585,2637,2645,2697,2726,2737,2789,2797,2849,2861,2878,2880,2881,2894,2898,2907,2912,2924,2946,2972,2973,3111,3123,3171,3239,3300,3473,3474,3483,3487,3490,3583,3584,3623,3635,3639,3647,3836,3839,3851,3855,3857,3858,3862,3867,3872,3881,3882,3965,3967,3972,3973,3976,3977,3978,3981,3990,3991,3992,3993,3994,3995

In [11]:
len(wide_df["record_id"].unique())

578

In [3]:
from scipy import stats

def print_statistics(df, column, group_by):
    # Overall statistics
    overall_mean = df[column].mean()
    overall_std = df[column].std()
    overall_non_na = df[column].notna().sum()

    print(f"Overall Statistics for {column}:")
    print(f"Mean: {overall_mean}")
    print(f"Standard Deviation: {overall_std}")
    print(f"Number of non-NA rows: {overall_non_na}")
    print("\n")

    # Grouped statistics
    grouped = df.groupby(group_by)[column]
    
    group_stats = grouped.agg(['mean', 'std', 'count']).reset_index()
    
    for _, row in group_stats.iterrows():
        print(f"Statistics for {column} in {group_by} = {row[group_by]}:")
        print(f"Mean: {row['mean']}")
        print(f"Standard Deviation: {row['std']}")
        print(f"Number of non-NA rows: {row['count']}")
        print("\n")
    
    # Bonferroni corrected p-value for the two groups
    groups = df[group_by].unique()
    if len(groups) == 2:
        group1 = df[df[group_by] == groups[0]][column].dropna()
        group2 = df[df[group_by] == groups[1]][column].dropna()

        t_stat, p_value = stats.ttest_ind(group1, group2)
        bonferroni_p_value = p_value * 2  # Bonferroni correction for two groups

        print(f"T-test between the two groups for {column}:")
        print(f"T-statistic: {t_stat}")
        print(f"Raw p-value: {p_value}")
        print(f"Bonferroni corrected p-value: {bonferroni_p_value}")
    else:
        print("More than two groups found. Bonferroni correction only applied for two groups.")

In [16]:
from scipy.stats import chi2_contingency

def print_categorical_statistics(df, column, group_by):
    # Overall statistics
    counts = df[column].value_counts(dropna=False)
    percentages = df[column].value_counts(normalize=True, dropna=False) * 100
    non_na_count = df[column].notna().sum()

    print(f"Overall Statistics for categorical variable {column}:")
    for value in counts.index:
        print(f"Value {value}: Count = {counts[value]}, Percentage = {percentages[value]:.1f}%")
    print(f"Number of non-NA rows: {non_na_count}")
    print("\n")

    # Grouped statistics
    grouped = df.groupby(group_by)[column]
    
    for group_value, group_df in grouped:
        group_counts = group_df.value_counts(dropna=False)
        group_percentages = group_df.value_counts(normalize=True, dropna=False) * 100
        group_non_na_count = group_df.notna().sum()

        print(f"Statistics for {column} in {group_by} = {group_value}:")
        for value in group_counts.index:
            print(f"Value {value}: Count = {group_counts[value]}, Percentage = {group_percentages[value]:.1f}%")
        print(f"Number of non-NA rows: {group_non_na_count}")
        print("\n")

    # Pearson's chi-square test
    contingency_table = pd.crosstab(df[group_by], df[column], dropna=False)
    chi2, p, _, _ = chi2_contingency(contingency_table)

    print(f"Pearson's Chi-square test for {column} by {group_by}:")
    print(f"Chi-square statistic: {chi2:.1f}")
    print(f"p-value: {p:.3f}")
    print("\n")

In [ ]:
age
resect_pt_age
sex_gender
age_of_onset_hw
freq_per_month_hw
eo_days_postsurg

In [ ]:
wide_df["age"]

In [13]:
print_statistics(wide_df, 'resect_pt_age', 'surg_engel_binary')

Overall Statistics for resect_pt_age:
Mean: 30.791522491349482
Standard Deviation: 15.891264425034162
Number of non-NA rows: 578


Statistics for resect_pt_age in surg_engel_binary = False:
Mean: 28.42292490118577
Standard Deviation: 15.954765799862589
Number of non-NA rows: 253


Statistics for resect_pt_age in surg_engel_binary = True:
Mean: 32.635384615384616
Standard Deviation: 15.618824021598547
Number of non-NA rows: 325


T-test between the two groups for resect_pt_age:
T-statistic: -3.186643992546755
Raw p-value: 0.0015172859751601713
Bonferroni corrected p-value: 0.0030345719503203426


In [17]:
print_categorical_statistics(wide_df, 'sex_gender', 'surg_engel_binary')

Overall Statistics for categorical variable sex_gender:
Value 0: Count = 295, Percentage = 51.0%
Value 1: Count = 283, Percentage = 49.0%
Number of non-NA rows: 578


Statistics for sex_gender in surg_engel_binary = False:
Value 1: Count = 130, Percentage = 51.4%
Value 0: Count = 123, Percentage = 48.6%
Number of non-NA rows: 253


Statistics for sex_gender in surg_engel_binary = True:
Value 0: Count = 172, Percentage = 52.9%
Value 1: Count = 153, Percentage = 47.1%
Number of non-NA rows: 325


Pearson's Chi-square test for sex_gender by surg_engel_binary:
Chi-square statistic: 0.9
p-value: 0.345


